# S1 JPEG, JPEG2000 and FFMpeg

In [1]:
!pip install pywavelets

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\22edu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import ffmpeg
from PIL import Image
from collections import OrderedDict
import pywt
import matplotlib.pyplot as plt
from scipy.fftpack import dct, idct



2) Start a script called first_seminar.py . Then create a class and a method , which is a translator from 3 values in RGB into the 3 YUV values, plus the opposite operation.

In [30]:
def RGB_to_YUV(rgb):
  m = np.array([[ 0.29900, -0.16874,  0.50000], [0.58700, -0.33126, -0.41869], [ 0.11400, 0.50000, -0.08131]])

  yuv = np.dot(rgb,m)
  yuv[:,:,1:]+=128.0

  return yuv

def YUV_to_RGB(yuv):
      
  m = np.array([[ 1.0, 1.0, 1.0], [-0.000007154783816076815, -0.3441331386566162, 1.7720025777816772], [ 1.4019975662231445, -0.7141380310058594 , 0.00001542569043522235]])
  
  rgb = np.dot(yuv,m)
  rgb[:,:,0]-=179.45477266423404
  rgb[:,:,1]+=135.45870971679688
  rgb[:,:,2]-=226.8183044444304
  return rgb

In [31]:
RGB_to_YUV(np.random.randint(0, 256, (100, 100, 3)))
YUV_to_RGB(np.random.randint(0, 256, (100, 100, 3)))

array([[[ -73.34602982,   71.15221524,   80.38694868],
        [  47.65705654,  -55.69424582,  223.18476183],
        [ 165.68351869,  258.48042274,  401.16332958],
        ...,
        [  26.58777849,   27.92843461,   93.47599251],
        [  57.72368466,  118.52165627,  212.00356076],
        [  25.49730848,   94.18986177,  299.00750716]],

       [[ 186.57753309,  186.11033177,  324.26800818],
        [ 185.52165121,   46.80185223,  149.61701315],
        [ 234.85365525,  164.29708076,  266.10851703],
        ...,
        [ 238.04364584,  -22.87790489,   81.17790256],
        [ -19.80705287,  233.65126967,  -45.88585432],
        [  81.97685212,  346.8582437 ,   95.46606352]],

       [[ 123.84441152,   99.65761733,  -22.17982819],
        [  88.06044929,   49.69757819,  -83.35572541],
        [  49.84480503,   44.58493996, -193.63996997],
        ...,
        [-168.64226359,   60.56831694,  139.21433456],
        [ 324.04371739,   66.56342649,  149.45787678],
        [ 104.79954221

3) Use ffmpeg to resize images into lower quality. Use any image you like

In [32]:
def resize_image(input_image, output_image, width, height):
    ffmpeg.input(input_image).output(output_image, vf=f'scale={width}:{height}').run(overwrite_output=True)

In [33]:
resize_image('image.png', 'image_resized.png', 50, 70)

4) Create a method called serpentine which should be able to read the bytes of a JPEG file in the serpentine way we saw.

In [34]:
def serpentine(img):
    rows, cols = img.shape
    idx = 0
    arr = []
    output = np.zeros([rows,cols], int)
    
    arr.append(img[0,0])

    for col in range(1, cols):
        i, j = 0, col   

        while i < rows and j >= 0:
            arr.append(img[i, j])
            i += 1
            j -= 1

    for row in range(1, rows):
        #i, j = row, cols - 1
        i, j = row, 0
        
        while j < cols and i >= 0:
            arr.append(img[i, j])
            i -= 1
            j += 1

    # Matrix
    for row in range(rows):
        for col in range(cols):
            output[row, col] = arr[idx]
            idx += 1
           


    return output

In [35]:
# Generate a 9x9 matrix with random bytes (values between 0 and 255)
image_matrix = np.random.randint(0, 256, size=(9, 9), dtype=np.uint8)

print("9x9 Image (Simulated Bytes):")
print(image_matrix)

rows, cols = image_matrix.shape

print("Serpentine Image")
print(serpentine(image_matrix))
            

9x9 Image (Simulated Bytes):
[[ 42 229  32  80 239  95  60 235  41]
 [ 78 162 134  49 150  51 157 129 233]
 [ 95 248 130 114  41 212 171 205 117]
 [246 212 142  35 193 162 158 215  55]
 [ 94 157 117  52 125  86 138  75  13]
 [103 136  96 211 196 191  59 251  92]
 [210  42 225 167 173  95 133 147 230]
 [ 38 183  87  51 183  39 230 230 146]
 [  4 215  92  41 248  49 167 253  95]]
Serpentine Image
[[ 42 229  78  32 162  95  80 134 248]
 [246 239  49 130 212  94  95 150 114]
 [142 157 103  60  51  41  35 117 136]
 [210 235 157 212 193  52  96  42  38]
 [ 41 129 171 162 125 211 225 183   4]
 [ 78 229  95 162  32 246 248 134  80]
 [ 94 212 130  49 239 103 157 142 114]
 [150  95 210 136 117  35  41  51  60]
 [ 38  42  96  52 193 212 157 235   4]]


5) Use FFMPEG to transform the previous image into b/w. Do the hardest compression you can.

In [36]:
# Using FFMPEG to transform an image into b/w
ffmpeg.input('DSC01760.JPEG').output('DSC01760_GRAY.JPEG', vf='format=gray').run(overwrite_output=True)

(None, None)

Comment de results:

- Per a llegir una imatge amb el patró de serpentina, hem utilitzat una matriu de bytes simulant una imatge en lloc d'una imatge real per facilitar els càlculs. Si ens fixem en el resultat obtingut, sembla que el patró de serpentina només funciona en sentit descendent, i no aplica la lectura en sentit ascendent. 

- Per a transformar una imatge JPEG en blanc i negre, fem servir una imatge real. Observem que la imatge s'ha transformat correctament a blanc i negre.

5) Create a method which applies a run-lenght encoding from a series of bytes given.

In [21]:
def runLengthEncoding(bytes):
    encoded=[]

    for i in range(len(bytes)):
        if bytes[i] != bytes[i - 1]:
            encoded.append(bytes[i])

    return encoded

In [24]:
# Testing run length encoding
# Test array with mixed patterns
data = [
    7, 7, 7,        # Three consecutive 7s
    8,              # Single 8
    9, 9, 9, 9, 9,  # Five consecutive 9s
    1, 2, 3,        # No repetitions
    4, 4, 4,        # Three consecutive 4s
    5, 6, 6,        # Two consecutive 6s
    7, 7,           # Two consecutive 7s
    8,              # Single 8
    9, 0, 0, 0      # Three consecutive 0s
]
print("Data:", data)

encoded_data = runLengthEncoding(data)
print("Encoded data with RLE:", encoded_data)


Data: [7, 7, 7, 8, 9, 9, 9, 9, 9, 1, 2, 3, 4, 4, 4, 5, 6, 6, 7, 7, 8, 9, 0, 0, 0]
Encoded data with RLE: [7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0]


6) Create a class which can convert, can decode (or both) an input using the DCT. Not necessary a JPG encoder or decoder. A class only about DCT is OK too

In [25]:
class DCTProcessor:
    def __init__(self, type=2, norm='ortho'):
        self.type = type
        self.norm = norm

    def encode(self, data):
        if data.ndim == 1:
            return dct(data, type=self.type, norm=self.norm)
        elif data.ndim == 2:
            return dct(dct(data.T, type=self.type, norm=self.norm).T, type=self.type, norm=self.norm)
        else:
            raise ValueError("Error")

    def decode(self, transformed_data):
        if transformed_data.ndim == 1:
            return idct(transformed_data, type=self.type, norm=self.norm)
        elif transformed_data.ndim == 2:
            return idct(idct(transformed_data.T, type=self.type, norm=self.norm).T, type=self.type, norm=self.norm)
        else:
            raise ValueError("Error")

    def process(self, data):
        encoded_data = self.encode(data)
        decoded_data = self.decode(encoded_data)
        return data, encoded_data, decoded_data

In [26]:
if __name__ == "__main__":
    # Create an instance of the DCTProcessor
    dct_processor = DCTProcessor(type=2, norm='ortho')

    # Test with 1D data
    data_1d = np.array([12, 34, 54, 23, 65, 23, 45, 76, 23, 56])
    
    # Test with 2D data
    data_2d = np.array([[12, 34, 54, 23],
                        [65, 23, 45, 76],
                        [23, 56, 12, 89],
                        [13, 45, 67, 32]])

    # Process the 1D data
    original_1d, encoded_1d, decoded_1d = dct_processor.process(data_1d)
    print("1D Data:")
    print("Original:", original_1d)
    print("Encoded (DCT):", encoded_1d)
    print("Decoded (IDCT):", decoded_1d)

    print("\n")

    # Process the 2D data
    original_2d, encoded_2d, decoded_2d = dct_processor.process(data_2d)
    print("2D Data:")
    print("Original:")
    print(original_2d)
    print("Encoded (DCT):")
    print(encoded_2d)
    print("Decoded (IDCT):")
    print(decoded_2d)

1D Data:
Original: [12 34 54 23 65 23 45 76 23 56]
Encoded (DCT): [129.96961183 -23.53734699 -11.3980292   -8.61584552 -18.97101233
  -4.11096096  -0.57873168 -39.02400824  34.47101233  15.04949448]
Decoded (IDCT): [12. 34. 54. 23. 65. 23. 45. 76. 23. 56.]


2D Data:
Original:
[[12 34 54 23]
 [65 23 45 76]
 [23 56 12 89]
 [13 45 67 32]]
Encoded (DCT):
[[167.25       -37.65653981  -0.75        -7.94418085]
 [ -7.18211348   8.65774708   8.4966421   16.25520382]
 [-27.25         6.69297723 -59.25        27.26406161]
 [-14.07274835 -10.24479618  -7.57839514 -37.65774708]]
Decoded (IDCT):
[[12. 34. 54. 23.]
 [65. 23. 45. 76.]
 [23. 56. 12. 89.]
 [13. 45. 67. 32.]]


7) Create a class which can convert, can decode (or both) an input using the DWT. Not necessary aJPEG2000 encoder or decoder. A class only about DWT is OK too

In [27]:
class DWTProcessor:
    def __init__(self, wavelet='haar', level=1):
        self.wavelet = wavelet
        self.level = level

    def encode(self, data):
        if data.ndim == 1:
            coeffs = pywt.wavedec(data, wavelet=self.wavelet, level=self.level)
            return coeffs
        elif data.ndim == 2:
            coeffs = pywt.dwt2(data, wavelet=self.wavelet)
            return coeffs
        else:
            raise ValueError("Error")

    def decode(self, transformed_data):
        if isinstance(transformed_data, list):
            data = pywt.waverec(transformed_data, wavelet=self.wavelet)
            return data
        elif isinstance(transformed_data, tuple):
            data = pywt.idwt2(transformed_data, wavelet=self.wavelet)
            return data
        else:
            raise ValueError("Error")

    def process(self, data):
        encoded_data = self.encode(data)
        decoded_data = self.decode(encoded_data)
        return data, encoded_data, decoded_data

In [28]:
if __name__ == "__main__":
    # Create an instance of the DWTProcessor
    dwt_processor = DWTProcessor(wavelet='haar', level=1)

    # Test with 1D data
    data_1d = np.array([12, 34, 54, 23, 65, 23, 45, 76, 23, 56])
    
    # Test with 2D data
    data_2d = np.array([[12, 34, 54, 23],
                        [65, 23, 45, 76],
                        [23, 56, 12, 89],
                        [13, 45, 67, 32]])

    # Process the 1D data
    original_1d, encoded_1d, decoded_1d = dwt_processor.process(data_1d)
    print("1D Data:")
    print("Original:", original_1d)
    print("Encoded (DWT):", encoded_1d)
    print("Decoded (IDWT):", decoded_1d)

    print("\n")

    # Process the 2D data
    original_2d, encoded_2d, decoded_2d = dwt_processor.process(data_2d)
    print("2D Data:")
    print("Original:")
    print(original_2d)
    print("Encoded (DWT):")
    print(encoded_2d)
    print("Decoded (IDWT):")
    print(decoded_2d)

1D Data:
Original: [12 34 54 23 65 23 45 76 23 56]
Encoded (DWT): [array([32.52691193, 54.44722215, 62.22539674, 85.55992052, 55.86143571]), array([-15.55634919,  21.92031022,  29.69848481, -21.92031022,
       -23.33452378])]
Decoded (IDWT): [12. 34. 54. 23. 65. 23. 45. 76. 23. 56.]


2D Data:
Original:
[[12 34 54 23]
 [65 23 45 76]
 [23 56 12 89]
 [13 45 67 32]]
Encoded (DWT):
(array([[ 67. ,  99. ],
       [ 68.5, 100. ]]), (array([[-21. , -22. ],
       [ 10.5,   1. ]]), array([[ 10. ,   0. ],
       [-32.5, -21. ]]), array([[-32. ,  31. ],
       [ -0.5, -56. ]])))
Decoded (IDWT):
[[12. 34. 54. 23.]
 [65. 23. 45. 76.]
 [23. 56. 12. 89.]
 [13. 45. 67. 32.]]
